In [2]:
#coding: utf-8

import warnings
warnings.simplefilter("ignore")

import os
import sys 
import wave
import time
import librosa
import numpy as np
import seaborn as sns
from utils import *
import matplotlib.pyplot as plt
from IPython.display import Audio
from StringIO import StringIO

%pylab inline

# sudo pip install librosa
# ubuntu: sudo apt-get install libav-tools
# OSX   : brew install libav OMP_NUM_THREADS=2

Populating the interactive namespace from numpy and matplotlib


# Get the Data

Датасет для контеста помещен вот здесь: TODO (внимание! файл весит 3,5 Гб)

Он распаковывается в папку "musicdata". Все пути в этом ноутбуке будут относительно этой папки: "./musicdata/*".

In [7]:
BP = "./musicdata/"

# Just Sound

Для начала попробуем поработать со звуком -- послушайте люббой трек, постройте график сырых данных и спектрограмму.

In [3]:
import IPython
from IPython import display

def Audio(url):
    return display.HTML("<center><audio controls><source src='{}'></audio>".format(url))

In [10]:
sound_file = './genres/blues/blues.00000.au'

y, sr = librosa.load(sound_file)
librosa.output.write_wav(os.path.join(BP, "audiosamples", "short_206024.wav"), y, sr, norm=True)
Audio(url='./genres/tmp.wav')

# Sound as 1D-Signal

In [ ]:
plt.figure(figsize=(20,4))
# 
# Используйте pylab.plot для построения графика, используйте y, sr
#
pylab.xlim([0, 10])
pylab.show()

# Sound as 2D-Signal

Нашу цель слелать из предыдущего графика картинку -- для] этого нам нужно оценить частоты в каждый момент времени; Блазо за нас это умеет делать librosa и внутри там STFFT используйте librosa.feature.melspectrogram -- для получения спектрограммы, и librosa.logamplitude для выраынивания диапозонов частот. 

help: https://gist.github.com/mailletf/3484932dd29d62b36092

In [ ]:
S = <librosa.feature.melspectrogram>
log_S = <librosa.logamplitude>

Нарисуйте спектрограмму, получилась красивая картинка?

In [ ]:
plt.figure(figsize=(20,4))
librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel', cmap='hot')
plt.title('mel power spectrogram')
plt.colorbar(format='%+02.0f dB')
plt.tight_layout()

# Prepare the Data

В этой секции мы создадим датасет, прочитав все данные.

In [1]:
import numpy as np
X = np.load(os.path.join(BP, "spectrograms", "spectrograms1.pcl"))
...

Здесь вы должны получить train genres. Они идут подряд в соответствии с последовательностью треков в файлах с данными. Для содержимого первых шести файлов у вас есть жанры, а для последнего файла, вам их нужно предсказать.

In [ ]:
with open(os.path.join(BP, "train_genres.txt"), "rt") as f_in:
    ...

In [ ]:
num_classes = len(set(train_genres))

# Nearest Neighbors genre classification

Let's try to solve simularity task by nearest neibor approuch.

<img src="./img/nn.png" width="300">

Перемешаем датасет -- используйте np.random.permutation и индексацию a[np.random.permutation].

In [ ]:
perm = np.random.permutation(len(y))
X, y = ????, ?????
Xreshape = X.reshape(X.shape[0], X.shape[1], X.shape[2])

Разобьем датасет на трейн и валидацию 80/20%.

In [ ]:
X_train, X_valid = Xreshape[:10000], Xreshape[10000:]
y_train, y_valid = y[:10000], y[10000:]

Попробуем поклассифицировать жанры методом ближайших соседей.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

clf = ????
clf = clf.fit???
y_val_pred = clf.predict(X_valid.reshape((X_valid.shape[0], -1)))

print accuracy_score(y_valid, y_val_pred)

# Fully-Connected Neural Nets

Использовать нейросети это хорошая идея, давайте начнем с полносвязных сетей.

In [ ]:
import theano
import lasagne
import theano.tensor as T

from utils import train_net

Перемешаем и разобьем датасет.

In [ ]:
perm = np.random.permutation(len(y))
X, y = np.array(X)[perm].astype('float32'), np.array(y)[perm]
Xreshape = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])

X_train, X_valid = Xreshape[:10000], Xreshape[10000:]
y_train, y_valid = y[:10000], y[10000:]

In [ ]:
input_X, target_y = T.tensor4("X", dtype='float32'), T.vector("y", dtype='int32')
nn = lasagne.layers.InputLayer(shape=(None, 1, X.shape[1], X.shape[2]), input_var=input_X) 

nn = ????
#
# Используйте несколько 2-3 больших полносвязных слоя, обучилась ли сеть? если нет то почему?
#

y_predicted = lasagne.layers.get_output(nn)
all_weights = lasagne.layers.get_all_params(nn)

loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()
updates_sgd = ???? выш любимый оптимизатор

In [ ]:
train_fun = theano.function([input_X, target_y], [loss, accuracy], allow_input_downcast=True, updates=updates_sgd)
test_fun  = theano.function([input_X, target_y], [loss, accuracy], allow_input_downcast=True)

In [ ]:
%time conv_nn = train_net(nn, train_fun, test_fun, X_train, y_train, X_valid, y_valid, num_epochs=10, batch_size=50)

# Convolution Neural Nets

Да, вероятно, полносвязные сети не зажгли и knn не обогнали. Дпвайте попробуем сверточные, хороший ваиант делать одномерные свертки (сразу по всем частотам). Хотя двумерные тоже будут работать. 

Архитектуру можно подсмотреть тут -- http://benanne.github.io/2014/08/05/spotify-cnns.html.

In [ ]:
import theano
import lasagne
import theano.tensor as T

from utils import train_net

In [ ]:
perm = np.random.permutation(len(y))
X, y = np.array(X)[perm].astype('float32'), np.array(y)[perm]
Xreshape = X.reshape(X.shape[0], X.shape[1], X.shape[2])

X_train, X_valid = Xreshape[:10000], Xreshape[10000:]
y_train, y_valid = y[:10000], y[10000:]

In [ ]:
input_X, target_y = T.tensor3("X", dtype='float64'), T.vector("y", dtype='int32')
nn = lasagne.layers.InputLayer(shape=(None, X.shape[1], X.shape[2]), input_var=input_X) 

nn = ??? Сделайте свою сеть используя -- Conv1DLayer + MaxPool1DLayer + DenseLayer

nn = lasagne.layers.DenseLayer(nn, num_classes, nonlinearity=lasagne.nonlinearities.softmax)

In [ ]:
y_predicted = lasagne.layers.get_output(nn)
all_weights = lasagne.layers.get_all_params(nn)

loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()
updates_sgd = lasagne.updates.adam(loss, all_weights)

In [ ]:
train_fun = theano.function([input_X, target_y], [loss, accuracy], allow_input_downcast=True, updates=updates_sgd)
test_fun  = theano.function([input_X, target_y], [loss, accuracy], allow_input_downcast=True)

In [ ]:
Подумайте -- какой размер батча нужен и сколько эпох авы можете себе позволить
%time conv_nn = train_net(nn, train_fun, test_fun, X_train, y_train, X_valid, y_valid, num_epochs=???, batch_size=??)

Валидационная точность должна быть около 0.6, или лучше. 

Давайте нарисуем каждый второй фильтр -- можете дли Вы проинтерпретировать эти фильтры?

In [ ]:
plt.figure(figsize=(5, 5), dpi=500)
W = lasagne.layers.get_all_params(nn)[0].get_value()
W[::2, :, :] = 0.2
W = np.hstack(W)
pylab.imshow(W, cmap='hot', interpolation="nearest")
pylab.axis('off')
pylab.show()

# Maps of tracks by svd and tsne

Класная идея посмотреть, как полученое представление оторажается с хохранением относительных расстояний на 2d. Испоьльзуйте TSNE ли PCA.

Help: https://lts2.epfl.ch/blog/perekres/category/visualizing-hidden-structures-in-datasets-using-deep-learning/

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
represent = lasagne.layers.get_output(nn.input_layer)
represent_fun = theano.function([input_X], [represent], allow_input_downcast=True)

In [ ]:
f = lambda x: np.array(represent_fun([x])[0])
track_vectors = map(f, X_train) + map(f, X_valid)
track_vectors = np.concatenate(track_vectors, axis=0)

track_labels = np.array(list(y_train) + list(y_valid))

получите 2d вектора

In [ ]:
X_tsne = ????

нарисуйте получившиеся точки

In [ ]:
plt.figure(figsize=(10,10), dpi=500)
colors = cm.hot(np.linspace(0, 1, len(id2gener)))

for idx, gener in id2gener.items():
    idx_ = np.where(track_labels == idx)
    pylab.scatter(????, ????, c=colors[idx], cmap=cm.hot, label=gener,s=50)

pylab.legend(loc=0, ncol=5)